# A2 Bias In Data

## TL;DR   
The purpose of this exploration is to identify potential sources of bias in a corpus of human-annotated data, and describe some implications of those biases.

_Data Source:  https://figshare.com/projects/Wikipedia_Talk/16731_  
_Overview of the project:  https://meta.wikimedia.org/wiki/Research:Detox_  
_Overview of the data: https://meta.wikimedia.org/wiki/Research:Detox/Data_Release_

## Aquire Data

Download all the data, using the `%%caption` magic keyword to suppress the outputs.

In [1]:
%%capture  

# download toxicity data
!wget https://ndownloader.figshare.com/files/7394539 -O Raw_Data/toxicity_annotations.tsv
!wget https://ndownloader.figshare.com/files/7394542 -O Raw_Data/toxicity_annotated_comments.tsv
!wget https://ndownloader.figshare.com/files/7640581 -O Raw_Data/toxicity_worker_demographics.tsv

# download aggression data
!wget https://ndownloader.figshare.com/files/7394506 -O Raw_Data/aggression_annotations.tsv
!wget https://ndownloader.figshare.com/files/7038038 -O Raw_Data/aggression_annotated_comments.tsv
!wget https://ndownloader.figshare.com/files/7640644 -O Raw_Data/aggression_worker_demographics.tsv
     
# downlaod personal attacks
!wget https://ndownloader.figshare.com/files/7554637 -O Raw_Data/attack_annotations.tsv
!wget https://ndownloader.figshare.com/files/7554634 -O Raw_Data/attack_annotated_comments.tsv
!wget https://ndownloader.figshare.com/files/7640752 -O Raw_Data/attack_worker_demographics.tsv


Read in the downloaded data as a pandas dataframe.

In [2]:
import pandas as pd

# toxicity data
toxicity_annotations = pd.read_csv("Raw_Data/toxicity_annotations.tsv", delimiter="\t")
toxicity_annotated_comments = pd.read_csv("Raw_Data/toxicity_annotated_comments.tsv", delimiter="\t", index_col=0)
toxicity_worker_demographics = pd.read_csv("Raw_Data/toxicity_worker_demographics.tsv", delimiter="\t")

# aggression data
aggression_annotations = pd.read_csv("Raw_Data/aggression_annotations.tsv", delimiter="\t")
aggression_annotated_comments = pd.read_csv("Raw_Data/aggression_annotated_comments.tsv", delimiter="\t", index_col=0)
aggression_worker_demographics = pd.read_csv("Raw_Data/aggression_worker_demographics.tsv", delimiter="\t")

# personal attack data
attack_annotations = pd.read_csv("Raw_Data/attack_annotations.tsv", delimiter="\t")
attack_annotated_comments = pd.read_csv("Raw_Data/attack_annotated_comments.tsv", delimiter="\t", index_col=0)
attack_worker_demographics = pd.read_csv("Raw_Data/attack_worker_demographics.tsv", delimiter="\t")

### DATA CLEANING

## Toxicity Analysis

First, examine the types of columns that typically get labeled as toxic.   We'll consider a comment to be toxic if at least 50% of reviewers labeled it as such.

In [15]:
# labels a comment as toxic if the majority of annotators did so
toxicity_labels = toxicity_annotations.groupby('rev_id')['toxicity'].mean() > 0.5

# join the labels and comments
toxicity_annotated_comments['labeled_toxic'] = toxicity_labels

# remove newline and tab tokens
toxicity_annotated_comments['comment'] = toxicity_annotated_comments['comment'].apply(lambda x: x.replace("NEWLINE_TOKEN", " "))
toxicity_annotated_comments['comment'] = toxicity_annotated_comments['comment'].apply(lambda x: x.replace("TAB_TOKEN", " "))

If we preview the results we can verify that these comments indeed appear to be toxic.

In [16]:
# preview the results
toxicity_annotated_comments.loc[toxicity_annotated_comments['toxic'] == True].head(15)

,comment,year,logged_in,ns,sample,split,toxic,labeled_toxic
rev_id,,,,,,,,
597212.0,"` After the wasted bit on his sexuality, I ha...",2003,False,article,random,test,True,True
1266286.0,"Erik, for crying out loud. You legally can...",2003,True,user,random,test,True,True
1502668.0,"BOOBS, BOOBS, BOOBS, BOOBS, BOOBS, BOOBS, BOOB...",2003,True,user,blocked,test,True,True
2187425.0,```Nazi filth`` is impolite `,2004,True,article,random,train,True,True
3129678.0,"Prior to Quickpolls, he would have been perma...",2004,True,user,random,train,True,True
3427333.0,"Hi. Yes, I'd noticed. It's always gratifing to...",2004,True,user,random,train,True,True
3803199.0,"Yep, Shiguy. That's me. David Hodges. I have n...",2004,True,user,random,train,True,True
4282271.0,Better believe it. At least I earned a Barn...,2004,True,user,random,train,True,True
4632658.0,"i have a dick, its bigger than yours! hahaha",2004,False,article,blocked,train,True,True


Since the data is divided into three separate files, it must be joined in various ways in order to analyze.

In [17]:
joined_toxicity_demographics = toxicity_annotations.join(toxicity_worker_demographics, on="worker_id", rsuffix="_r")
joined_toxicity_comments = toxicity_annotations.join(toxicity_annotated_comments, on="rev_id", rsuffix="_r")

Quickly previewing the results of each table:

In [18]:
display(joined_toxicity_demographics.head())
display(joined_toxicity_comments.head())

,rev_id,worker_id,toxicity,toxicity_score,worker_id_r,gender,english_first_language,age_group,education
0,2232.0,723,0,0.0,1789.0,male,1.0,30-45,bachelors
1,2232.0,4000,0,0.0,NaN,NaN,NaN,NaN,NaN
2,2232.0,3989,0,1.0,NaN,NaN,NaN,NaN,NaN
3,2232.0,3341,0,0.0,3974.0,male,0.0,18-30,hs
4,2232.0,1574,0,1.0,3863.0,female,0.0,18-30,professional


,rev_id,worker_id,toxicity,toxicity_score,comment,year,logged_in,ns,sample,split,toxic,labeled_toxic
0,2232.0,723,0,0.0,This: :One can make an analogy in mathematical...,2002,True,article,random,train,False,False
1,2232.0,4000,0,0.0,This: :One can make an analogy in mathematical...,2002,True,article,random,train,False,False
2,2232.0,3989,0,1.0,This: :One can make an analogy in mathematical...,2002,True,article,random,train,False,False
3,2232.0,3341,0,0.0,This: :One can make an analogy in mathematical...,2002,True,article,random,train,False,False
4,2232.0,1574,0,1.0,This: :One can make an analogy in mathematical...,2002,True,article,random,train,False,False


### Personal Attack Analysis

In [ ]:
# labels a comment as an atack if the majority of annoatators did so
labels = attack_annotations.groupby('rev_id')['attack'].mean() > 0.5

# join labels and comments
attack_annotated_comments['attack'] = labels

Join the dataframes from similar datasets in order to analyze as a whole.

In [ ]:
joined_toxicity_annotations = toxicity_annotations.join(toxicity_worker_demographics, on="worker_id", rsuffix="_r")
joined_aggression_annotations = aggression_annotations.join(aggression_worker_demographics, on="worker_id", rsuffix="_r")
joined_attack_annotations = attack_annotations.join(attack_worker_demographics, on="worker_id", rsuffix="_r")

In [ ]:
joined_toxicity_annotations = toxicity_annotations.join(toxicity_annotated_comments, on="rev_id", rsuffix="_r")


In [ ]:

# labels a comment as an atack if the majority of annoatators did so
labels = attack_annotations.groupby('rev_id')['attack'].mean() > 0.5

In [ ]:

# join labels and comments
attack_annotated_comments['attack'] = labels

In [ ]:
# remove newline and tab tokens
attack_annotated_comments['comment'] = attack_annotated_comments['comment'].apply(lambda x: x.replace("NEWLINE_TOKEN", " "))
attack_annotated_comments['comment'] = attack_annotated_comments['comment'].apply(lambda x: x.replace("TAB_TOKEN", " "))

attack_annotated_comments.loc[attack_annotated_comments['attack'] == True]

Preview the results.

In [ ]:
joined_toxicity_annotations.head()

## Analysis / Visualization

Calculate the average score per worker - that worker's "toxicity bias". Is this different for different age groups?

In [ ]:
avg_worker_toxicity = joined_toxicity_annotations.groupby("worker_id")["toxicity_score"].mean()
toxicity_worker_demographics = toxicity_worker_demographics.join(avg_worker_toxicity)
toxicity_worker_demographics.head()


Now let's compute an average toxicity statistic for each group...

In [ ]:
toxicity_worker_demographics.groupby("age_group").toxicity_score.mean()

## Implications

### 

---

In [ ]:
# requires plotly install
# conda install -c anaconda plotly
import plotly.express as px



# also needs ipython widgets
conda install jupyterlab "ipywidgets=7.5"


# JupyterLab renderer support
jupyter labextension install jupyterlab-plotly@4.11.0

jupyter labextension install @jupyter-widgets/jupyterlab-manager plotlywidget@4.11.0

In [ ]:
import plotly.graph_objects as go
fig = go.FigureWidget(data=go.Bar(y=[2, 3, 1]))
fig

In [ ]:
import plotly.graph_objects as go
fig = go.Figure(data=go.Bar(y=[2, 3, 1]))
fig.write_html('second_figure.html', auto_open=True)

In [ ]:
import plotly.graph_objects as go
fig = go.Figure(data=go.Bar(y=[2, 3, 1]))
fig.show()

In [ ]:
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt